In [9]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

from config import API_KEY_INFO
service_key = API_KEY_INFO.get("ecos")

# Test

In [17]:
from PublicDataReader import Ecos
# service_key = "API 인증키"
api = Ecos(service_key)
df = api.get_statistic_table_list()
df.head()

,상위통계표코드,통계표코드,통계명,시점,검색가능여부,출처
0,*,0000000001,1. 통화/금융,None,N,None
1,0000000001,0000000002,1.1. 통화/유동성,None,N,None
2,0000000002,0000000003,1.1.1. 본원통화,None,N,None
3,0000000003,0000000004,1.1.1.1. 본원통화 구성내역,None,N,None
4,0000000004,102Y004,"1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)",M,Y,None


In [12]:
from PublicDataReader import Ecos
# service_key = "API 인증키"
api = Ecos(service_key)
df = api.get_statistic_word(용어="소비자동향지수")
df.head()

,용어,용어설명
0,소비자동향지수,"소비자들이 느끼는 경기, 소비지출계획, 생활형편 등 경제에 대한 전반적인 인식을 조..."


In [13]:
from PublicDataReader import Ecos
# service_key = "API 인증키"
api = Ecos(service_key)
df = api.get_statistic_item_list(통계표코드="601Y002")
df.head()

,통계표코드,통계명,항목그룹코드,항목그룹명,통계항목코드,통계항목명,상위통계항목코드,상위통계항목명,시점,수록시작일자,수록종료일자,자료수,단위,가중치
0,601Y002,7.5.2. 지역별 소비유형별 개인 신용카드,Group1,지역코드,A,서울,None,None,M,200912,202212,12874,None,None
1,601Y002,7.5.2. 지역별 소비유형별 개인 신용카드,Group1,지역코드,B,부산,None,None,M,200912,202212,12874,None,None
2,601Y002,7.5.2. 지역별 소비유형별 개인 신용카드,Group1,지역코드,C,대구,None,None,M,200912,202212,12874,None,None
3,601Y002,7.5.2. 지역별 소비유형별 개인 신용카드,Group1,지역코드,D,인천,None,None,M,200912,202212,12874,None,None
4,601Y002,7.5.2. 지역별 소비유형별 개인 신용카드,Group1,지역코드,E,광주,None,None,M,200912,202212,12874,None,None


In [14]:
from PublicDataReader import Ecos
# service_key = "API 인증키"
api = Ecos(service_key)
df = api.get_statistic_search(통계표코드="200Y001", 주기="A", 검색시작일자="2015", 검색종료일자="2021")
df.head()

,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,통계항목코드4,통계항목명4,단위,시점,값
0,200Y001,2.1.1.1. 주요지표(연간지표),3010101,민간,None,None,None,None,None,None,%,2015,2.2
1,200Y001,2.1.1.1. 주요지표(연간지표),7010106,가계,None,None,None,None,None,None,%,2015,55.8
2,200Y001,2.1.1.1. 주요지표(연간지표),7010107,기업,None,None,None,None,None,None,%,2015,22.5
3,200Y001,2.1.1.1. 주요지표(연간지표),7010108,정부,None,None,None,None,None,None,%,2015,21.7
4,200Y001,2.1.1.1. 주요지표(연간지표),70105,총조정처분가능소득의분배,None,None,None,None,None,None,None,2015,None


In [15]:
from PublicDataReader import Ecos
# service_key = "API 인증키"
api = Ecos(service_key)
df = api.get_key_statistic_list()
df.head()

,통계그룹명,통계명,값,시점,단위
0,시장금리,한국은행 기준금리,3.5,20230315,%
1,시장금리,콜금리(익일물),3.439,20230316,%
2,시장금리,KORIBOR(3개월),3.56,20230317,%
3,시장금리,CD수익률(91일),3.62,20230317,%
4,시장금리,통안증권수익률(364일),3.429,20230317,%


In [16]:
from PublicDataReader import Ecos
# service_key = "API 인증키"
api = Ecos(service_key)
df = api.get_statistic_meta(데이터명="경제심리지수")
df.head()

,레벨,상위통계항목코드,통계항목코드,통계항목명,메타데이터
0,2,0000000001,0000000098,무응답률,None
1,2,0000000001,0000000099,MSE,None
2,1,None,0000000100,예산 인력 교육 홍보,None
3,2,0000000001,0000000101,관련 예산,None
4,2,0000000001,0000000102,인력,None


# Develop

In [11]:
import requests
import pandas as pd

requests.packages.urllib3.disable_warnings()